In [1]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import datetime

In [ ]:
GPS_DATA_FOLDER = Path("../BDC_30_routes_April_2025/raw_GPS")
CLEAN_GPS_DATA_FOLDER = Path("../BDC_30_routes_April_2025/clean_GPS")

In [ ]:
DATA_PATH = Path("../BDC_30_routes_April_2025/raw_GPS/anonymized_raw_2025-04-01.csv")
#DATA_PATH = Path("/kaggle/input/bdc-hakathon/raw_GPS/anonymized_raw_2025-04-01.csv")

if not DATA_PATH.is_file():
    raise FileNotFoundError(f"Không tìm thấy file {DATA_PATH.resolve()}")

gps_df = pd.read_csv(DATA_PATH, low_memory=False)
print(f"Đang đọc: {DATA_PATH.resolve()}")
print(f"Số bản ghi: {gps_df.shape[0]:,} | Số cột: {gps_df.shape[1]}")
gps_df.head()

Đang đọc: C:\Users\likgn\Repository\BDC_Data\BDC_30_routes_April_2025\raw_GPS\anonymized_raw_2025-04-01.csv
Số bản ghi: 2,354,942 | Số cột: 8


,datetime,lng,lat,speed,door_up,door_down,anonymized_vehicle,anonymized_driver
0,2025-04-01 00:00:01,106.482285,10.971292,NaN,False,False,f3a3e75584,NaN
1,2025-04-01 00:00:10,106.705688,10.694684,NaN,False,False,7121f294ef,NaN
2,2025-04-01 00:00:28,106.512432,10.787085,NaN,False,False,6cd3c92c48,NaN
3,2025-04-01 00:00:01,106.741605,10.602732,NaN,False,False,f373b0e01d,NaN
4,2025-04-01 00:00:01,106.640020,10.769893,NaN,False,False,058ff527c8,NaN


In [4]:
vehicle_col = "anonymized_vehicle"
driver_col = "anonymized_driver"
lat_col = "lat"
lng_col = "lng"
time_col = "datetime"

# Tiền xử lý dữ liệu

In [25]:
# Chuyển đổi loại cột
gps_df.loc[:, time_col] = pd.to_datetime(gps_df.loc[:, time_col])
gps_df.loc[:, lat_col] = pd.to_numeric(gps_df.loc[:, lat_col], errors='coerce')
gps_df.loc[:, lng_col] = pd.to_numeric(gps_df.loc[:, lng_col], errors='coerce')

In [26]:
# Remove duplicate rows
print(f"Số bản ghi ban đầu: {len(gps_df)}")
gps_df = gps_df.drop_duplicates()
print(f"Sau khi loại bỏ bản ghi trùng lặp: Số bản ghi: {len(gps_df)}")

Số bản ghi ban đầu: 2343951
Sau khi loại bỏ bản ghi trùng lặp: Số bản ghi: 2343951
Sau khi loại bỏ bản ghi trùng lặp: Số bản ghi: 2343951


In [27]:
# Chỉ lọc ra một khoảng thời gian trong ngày
selected_time_window = (datetime.datetime(2025, 4, 1, 5, 0, 0), datetime.datetime(2025, 4, 1, 23, 0, 0))
gps_df = gps_df.loc[(gps_df[time_col] >= selected_time_window[0]) & (gps_df[time_col] <= selected_time_window[1])]
print(f"Số bản ghi sau khi lọc theo khoảng thời gian {selected_time_window[0]} đến {selected_time_window[1]}: {len(gps_df)}")

Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-01 05:00:00 đến 2025-04-01 23:00:00: 2056887


In [53]:
def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """Apply all cleaning and transformation steps for a single raw GPS dataframe."""
    processed = df.copy()

    # Chuyển đổi kiểu dữ liệu
    processed.loc[:, time_col] = pd.to_datetime(processed.loc[:, time_col])
    processed.loc[:, lat_col] = pd.to_numeric(processed.loc[:, lat_col], errors="coerce")
    processed.loc[:, lng_col] = pd.to_numeric(processed.loc[:, lng_col], errors="coerce")

    print("Số lượng bản ghi ban đầu:", len(df))
    processed = processed.drop_duplicates()
    print("Số lượng bản ghi sau khi loại bỏ trùng lặp:", len(processed))

    # Gioi hạn thoi gian hoạt động của xe trong ngày từ 5h sáng đến 23h tối
    selected_time_window = (datetime.datetime(2025, 4, 1, 5, 0, 0), datetime.datetime(2025, 4, 1, 23, 0, 0))
    processed = processed.loc[(processed[time_col] >= selected_time_window[0]) & (processed[time_col] <= selected_time_window[1])]
    print(f"Số bản ghi sau khi lọc theo khoảng thời gian {selected_time_window[0]} đến {selected_time_window[1]}: {len(processed)}")

    # Sắp xếp lại dữ liệu theo từng xe và thời gian ghi nhận
    processed = processed.sort_values(by=[vehicle_col, driver_col, time_col], ascending=[True, True, True])

    # Giảm số lượng lấy mẫu, mỗi phút chỉ lấy một mẫu (giữ record đầu tiên trong phút)
    processed["time_minute"] = processed[time_col].dt.floor("1min") # Tạo cột mới theo thời gian chỉ lấy đến phút
    processed = processed.set_index("time_minute")
    resampled_frames = []
    agg_dict = {
        lat_col: "first",
        lng_col: "first",
        "speed": "first",
        "door_up": lambda x: x.astype(bool).any(),
        "door_down": lambda x: x.astype(bool).any(),
        vehicle_col: "first",
        driver_col: "first",
    }

    for vehicle_id, group in processed.groupby([vehicle_col]):
        group = group.sort_index()
        resampled = group.resample("1min").agg(agg_dict).dropna(subset=[lat_col, lng_col])
        resampled_frames.append(resampled.reset_index())

    processed = pd.concat(resampled_frames, ignore_index=True)
    processed["door_up"] = processed["door_up"].astype(bool)
    processed["door_down"] = processed["door_down"].astype(bool)
    
    print("Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu:", len(processed))
    return processed


In [38]:
min_gps_sample_vehicle = "d62a133ba6"
gps_df[gps_df.loc[:, vehicle_col] ==  min_gps_sample_vehicle].sort_values(by=time_col, axis="index")

,datetime,lng,lat,speed,door_up,door_down,anonymized_vehicle,anonymized_driver
328363,2025-04-01 05:44:37,106.817382,10.807168,NaN,False,False,d62a133ba6,NaN
329321,2025-04-01 05:44:57,106.817382,10.807168,NaN,False,False,d62a133ba6,NaN
327660,2025-04-01 05:45:17,106.817458,10.807172,NaN,False,False,d62a133ba6,NaN
327846,2025-04-01 05:45:37,106.817458,10.807172,NaN,False,False,d62a133ba6,NaN
328995,2025-04-01 05:45:57,106.817458,10.807172,NaN,False,False,d62a133ba6,NaN
...,...,...,...,...,...,...,...,...
2245582,2025-04-01 21:44:58,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN
2246339,2025-04-01 21:45:18,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN
2246301,2025-04-01 21:45:38,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN
2246554,2025-04-01 21:45:58,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN


In [54]:
process_gps_df = process_data(gps_df)

Số lượng bản ghi ban đầu: 2056887
Số lượng bản ghi sau khi loại bỏ trùng lặp: 2056887
Số bản ghi sau khi lọc theo khoảng thời gian 2025-04-01 05:00:00 đến 2025-04-01 23:00:00: 2056887
Số lượng bản ghi sau khi giảm lấy mẫu mỗi phút một mẫu: 459944


In [55]:
process_gps_df["anonymized_vehicle"].nunique()

459

# Khám phá dữ liệu GPS xe buýt
- Trích rút thống kê cơ bản (số xe, số tài xế, thời gian ghi nhận)

In [28]:
# Tóm tắt về số lượng chyến xe và số lượng tài xế
summary = pd.Series({
    "Số dòng": len(gps_df),
    "Số xe duy nhất": gps_df[vehicle_col].nunique(dropna=True),
    "Số tài xế duy nhất": gps_df[driver_col].nunique(dropna=True),
})

time_window = gps_df[time_col].agg(["min", "max"])
summary["Thời gian bắt đầu"] = time_window["min"]
summary["Thời gian kết thúc"] = time_window["max"]

display(summary.to_frame(name="Giá trị"))

,Giá trị
Số dòng,2056887
Số xe duy nhất,459
Số tài xế duy nhất,276
Thời gian bắt đầu,2025-04-01 05:00:00
Thời gian kết thúc,2025-04-01 23:00:00


- Kiểm tra số lần lấy mẫu gps của từng xe

In [29]:
number_gps_sample = gps_df.loc[:, [time_col, vehicle_col]].groupby(vehicle_col).count()
number_gps_sample = number_gps_sample.rename(columns={time_col: "số_lần_lấy_mẫu_gps"}).sort_values(by="số_lần_lấy_mẫu_gps", ascending=False) # Rename count column
number_gps_sample

,số_lần_lấy_mẫu_gps
anonymized_vehicle,
7bc60c1f37,12960
90b8c44914,6517
fe41234204,6474
27c55c00eb,6471
627734efaf,6469
...,...
bfd1d69ad7,2102
6a7a034818,2060
7a53636e3d,1975


- Kiểm tra trường hợp dữ liệu cao nhất và thấp nhất

In [30]:
min_gps_sample_vehicle = "d62a133ba6"
gps_df[gps_df.loc[:, vehicle_col] ==  min_gps_sample_vehicle].sort_values(by=time_col, axis="index")

,datetime,lng,lat,speed,door_up,door_down,anonymized_vehicle,anonymized_driver
328363,2025-04-01 05:44:37,106.817382,10.807168,NaN,False,False,d62a133ba6,NaN
329321,2025-04-01 05:44:57,106.817382,10.807168,NaN,False,False,d62a133ba6,NaN
327660,2025-04-01 05:45:17,106.817458,10.807172,NaN,False,False,d62a133ba6,NaN
327846,2025-04-01 05:45:37,106.817458,10.807172,NaN,False,False,d62a133ba6,NaN
328995,2025-04-01 05:45:57,106.817458,10.807172,NaN,False,False,d62a133ba6,NaN
...,...,...,...,...,...,...,...,...
2245582,2025-04-01 21:44:58,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN
2246339,2025-04-01 21:45:18,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN
2246301,2025-04-01 21:45:38,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN
2246554,2025-04-01 21:45:58,106.817400,10.806985,NaN,False,False,d62a133ba6,NaN


In [33]:
max_gps_sample_vehicle = "7bc60c1f37"
gps_df[gps_df.loc[:, vehicle_col] ==  max_gps_sample_vehicle].sort_values(by=time_col, axis="index")

,datetime,lng,lat,speed,door_up,door_down,anonymized_vehicle,anonymized_driver
244855,2025-04-01 05:00:01,106.78067,10.777943,NaN,False,False,7bc60c1f37,NaN
243312,2025-04-01 05:00:06,106.78067,10.777943,NaN,False,False,7bc60c1f37,NaN
243843,2025-04-01 05:00:11,106.78067,10.777943,NaN,False,False,7bc60c1f37,NaN
244479,2025-04-01 05:00:16,106.78067,10.777943,NaN,False,False,7bc60c1f37,NaN
244568,2025-04-01 05:00:21,106.78067,10.777943,NaN,False,False,7bc60c1f37,NaN
...,...,...,...,...,...,...,...,...
2309789,2025-04-01 22:59:38,106.78075,10.777977,NaN,False,False,7bc60c1f37,NaN
2309791,2025-04-01 22:59:43,106.78075,10.777977,NaN,False,False,7bc60c1f37,NaN
2309276,2025-04-01 22:59:48,106.78075,10.777977,NaN,False,False,7bc60c1f37,NaN
2309190,2025-04-01 22:59:53,106.78075,10.777977,NaN,False,False,7bc60c1f37,NaN


# Định hướng xử lý dữ liệu

## Lọc một chuyến và vẽ lên bản đồ
Chọn ngẫu nhiên một `anonymized_vehicle`, sau đó vẽ toàn bộ tọa độ của xe lên bản đồ để kiểm tra quãng đường đã ghi nhận trong ngày.

In [7]:
# Lấy ngẫu nhiên một xe
vehicle_values = gps_df[vehicle_col].dropna().unique()
vehicle_sample = pd.Series(vehicle_values).sample(1, random_state=42).iloc[0]

# Trích xuất dữ liệu gps của một xe
vehicle_track = gps_df[gps_df[vehicle_col] == vehicle_sample].copy()
vehicle_track[lat_col] = pd.to_numeric(vehicle_track[lat_col], errors="coerce")
vehicle_track[lng_col] = pd.to_numeric(vehicle_track[lng_col], errors="coerce")
vehicle_track[time_col] = pd.to_datetime(vehicle_track[time_col], errors="coerce")
vehicle_track = vehicle_track.dropna(subset=[lat_col, lng_col])

if time_col:
    vehicle_track = vehicle_track.sort_values(time_col)


# Chỉ lọc ra một khoảng thời gian trong ngày
selected_time_window = (datetime.datetime(2025, 4, 1, 0, 0, 0), datetime.datetime(2025, 4, 1, 12, 0, 0))
vehicle_track = vehicle_track.loc[(vehicle_track[time_col] >= selected_time_window[0]) & (vehicle_track[time_col] <= selected_time_window[1])]
print(f"Xe được chọn: {vehicle_sample} | Số điểm GPS hợp lệ: {len(vehicle_track)}")

plot_kwargs = {
    "data_frame": vehicle_track,
    "lat": lat_col,
    "lon": lng_col,
    "hover_name": vehicle_col,
    "hover_data": [c for c in [driver_col, time_col] if c],
    "zoom": 11,
    "height": 600
    
}

plot_kwargs["color"] = time_col
plot_kwargs["color_continuous_scale"] = "Turbo"


Xe được chọn: bb4a866a49 | Số điểm GPS hợp lệ: 1774


In [ ]:
fig = px.scatter_mapbox(**plot_kwargs)
fig.update_layout(mapbox_style="carto-positron", title=f"Quỹ đạo xe {vehicle_sample}")
fig